# 4. 신경망 학습

학습은 훈련 데이터를 통해 최적의 가중치 매개변수를 자동으로 획득하는 것을 의미한다. 

## 4-1. 데이터와 데이터 주도 학습

사람이 직접 알고리즘 짜지 않고 데이터로부터 패턴을 발견하는 것이 기계 학습이다. 예컨대 특정 숫자의 특징을 이미지로부터 사람이 직접 추출하고, 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있다. 다만 이 때도 이미지를 벡터로 변환할 때 사용하는 특징은 사람이 설계(SIFT, HOG 등)해야 한다. 신경망(딥러닝)은 이와 달리 end-to-end machine learning이기 때문에 이미지에 포함된 중요한 특징까지도 기계가 스스로 학습한다.

이 때, 사용되는 데이터는 '훈련 데이터'와 '시험 데이터'로 나뉜다. 이렇게 목적에 따라 데이터를 나눔으로써 신경망의 범용적 성능을 알 수 있게 된다. 범용적 성능을 가지고 있지 않은 신경망은, 예를 들어 특정 인물의 글씨체(특정 데이터셋)만 더욱 잘 인식한다거나 하는 등의 '오버피팅(overfitting)'에 직면할 수 있다. 

## 4-2. 손실 함수

'신경망의 성능이 얼마나 나쁜가'의 척도. 신경망은 이 손실 함수를 기준으로 최적의 매개변수 값을 탐색한다. 손실함수는 일반적으로 오차제곱합과 교차 엔트로피 오차를 사용한다.

중요한 점은 신경망을 학습할 때는 정확도를 지표로 삼을 수 없다는 것이다. 정확도를 지표로 삼으면 대부분의 장소에서 매개변수(편향치와 가중치)의 미분 값이 0이 되기 때문이다.

### 4-2-1. 오차제곱합 (Sum of Squres for Error, SEE)

말 그대로 오차의 제곱 합이다. 정답 레이블을 $t$, 신경망의 출력(신경망이 추정한 값)을 $y$라 할 때, SSE는 다음과 같다.


$$\frac{1}{2}\sum_k(y_k-t_k)^2$$

예를 들어, 이미지를 입력으로 받아 숫자를 추정하는 프로그램이라고 하면, 출력은 0 ~ 9로 총 1x10 행렬일 것이다. 정답은 10개 중 하나일 것이므로 단 하나만 1 로 출력되어 '원-핫 인코딩'이 될 것이고, 신경망의 출력과 정답 레이블은 각각

```python
y = [0.1, 0.7, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0] # 신경망의 출력. 소프트맥스 함수이므로 각 숫자일 확률로 볼 수 있다.
t = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0] # 정답은 1
```
과 같이 될 것이다. 소프트맥스 함수의 출력값과 정답 레이블의 오차가 커질수록 SSE 함수의 출력도 커질 것이므로 이는 '신경망의 나쁨의 정도'로 적절히 사용될 수 있다.

### 4-2-2. 교차 엔트로피 오차 (Cross Entropy Error, CEE)

$$E = -\sum_kt_k\log y_k$$

이 때 정답 레이블 $t_k$는 오직 정답에 대해서만 1이고 나머지는 0인 원-핫 인코딩이므로, 결국 정답 레이블과 같은 인덱스의 출력 $y_k$가 1.0에 다가갈수록 0에 가까워지고, 0.0에 가까워질수록 1에 가까워져 오차의 정도를 계산할 수 있게 된다. 코드로 쓰면 아래와 같다.

In [1]:
import numpy as np

def calculate_single_cee(y : np.array, t : np.array) -> float :
    delta = 1e-7
    return  -1 * np.sum(t * np.log(y + delta))

print(calculate_single_cee(np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), np.array([0, 0, 0.5, 0.2, 0, 0, 0.1, 0.2, 0, 0])))
print(calculate_single_cee(np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), np.array([0, 0.2, 0.1, 0.2, 0, 0, 0.1, 0.2, 0.1, 0.1])))


8.059047775479163
14.50628607586249


이 때, 로그 함수는 점근선이 $x = 0$ 이므로 아주 작은 값 delta를 더하여 Nan 출력을 방지해준다.

지금까지 구한 손실함수는 데이터 1개에 대한 손실함수이므로, N 개의 데이터에 대해 교차 엔트로피 오차를 구하면, 

$$E = -\frac{1}{N}\sum_n \sum_kt_{nk} \log y_{nk}$$

로 정규화할 수 있다. 이렇게 정규화함으로써 데이터의 크기가 1000이든 100이든 통일된 오차 지표를 구할 수 있다.

그런데 보통 데이터는 매우 많으므로, 일반적으로는 랜덤으로 조금씩 빼서 손실 함수를 구하고 학습하는데, 이러한 학습 방법을 미니배치 학습이라고 한다.

In [2]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label = True)

print(x_train.shape) # 28 * 28 = 784
print(t_train.shape)

train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)
print(batch_mask, ", type : ", type(batch_mask))
x_train_batch = x_train[batch_mask]
print("x_train_batch.shape : ", x_train_batch.shape)
t_train_batch = t_train[batch_mask]
print("t_train_batch.shape : ", t_train_batch.shape)

(60000, 784)
(60000, 10)
[39030 18142 54887  8893 27707 10719 58685 25576  7171 13417] , type :  <class 'numpy.ndarray'>
x_train_batch.shape :  (10, 784)
t_train_batch.shape :  (10, 10)


이를 바탕으로 10개의 데이터에 대해 교차 엔트로피 에러를 구해보면 아래와 같다.

In [3]:
import numpy as np

def calculate_cee(y : np.array, t : np.array) -> float :
    if y.ndim == 1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    delta = 1e-7
    return -1 * np.sum(t * np.log(y + delta)) / batch_size

## 4-3. 수치미분

간단한 수치 미분의 예시는 아래와 같다. 대학교 졸업논문 작성때 배웠던 내용과 같다. 간단하게 아래의 수식을 코드로 구현하면 되는데, 중요한 점은 precision issue가 생길 수 있으므로 h의 값은 1e-4로 하자.
전방 차분은

$$\lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$$

오차를 줄이기 위해 중앙 차분을 구하면,

$$\lim_{h \to 0} \frac{f(x + h) - f(x - h)}{2h}$$

In [4]:
def simple_func(x) :
    return x*x + 3*x + 1

def numerical_diff(f, x) :
    h = 1e-4 # precision issue에 조심한다.
    return (f(x + h) - f(x - h)) / 2*h

print(numerical_diff(simple_func, 2)) # 해석적으로 풀면 7


7.000000000019214e-08


### 4-3-1. 편미분과 기울기(gradient)

다변수 함수의 특정 변수에 대한 편미분은 각각의 변수에 대해서만 시행하면 된다. 기울기(gradient)는 모든 변수에 대한 편미분을 벡터로 정리한 것이다. 예를 들어, 아래의 함수가 있다고 해보자.

$$f(x_0, x_1) = x_0^2 + x_1^2$$

이 때, 각 변수에 대한 편도함수는

$$f_{x_0} = 2x_0$$
$$f_{x_1} = 2x_1$$

수치 미분을 적용하여 기울기를 코드로 작성하면 아래와 같다.

In [5]:
def numerical_gradient(f, x) :
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size) :
        temp_val = x[idx]
        # f(x + h)
        x[idx] = temp_val + h 
        fxh1 = f(x)
        
        # f(x - h)
        x[idx] = temp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = temp_val
        
    return grad

이제 numerical_gradient 함수를 이용해서 gradient descent를 구현해본다. gradient descent를 수식으로 나타내면,

$$x_0' = x_0 - \eta \frac{\delta f}{\delta x_0}$$

$$x_1' = x_1 - \eta \frac{\delta f}{\delta x_1}$$

따라서, 이를 기반으로 코드를 작성하면

In [6]:
def gradient_descent(f, init_x, step = 0.1, itr = 100) :
    x = init_x
    
    for i in range(itr) :
        grad = numerical_gradient(f, x)
        x -= step * grad
    return x

def func_example(x) :
    return x[0]**2 + x[1]**2

print(gradient_descent(func_example, np.array([4.0, 3.0])))

print(gradient_descent(func_example, np.array([4.0, 3.0]), step = 0.0001)) # step size가 너무 작다면? -> x wouldn't converge well.

print(gradient_descent(func_example, np.array([4.0, 3.0]), step = 10)) # step size가 너무 크다면? -> x would diverge.

[8.14814391e-10 6.11110793e-10]
[3.92078685 2.94059014]
[-1.29524862e+12  2.58983747e+13]


위의 내용을 바탕으로 확률적 경사 하강법(Stochastic Gradient Descent)를 구현해보자. '확률적'이라는 뜻은, 미니 배칠를 통해 일부 데이터만의 경사 하강을 구하기 때문에 붙은 이름이다.

In [7]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
# from common.gradient import numerical_gradient
import common.gradient
from dataset.mnist import load_mnist

class TwoLayerNet : # 하나의 은닉층과 하나의 출력층으로 구성되어 있는 네트워크.
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01) :
        
        #params 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x) :
        
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1) # 은닉층의 출력. 시그모이드 함수.
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2) # 출력층. 소프트맥스 함수.
        
        return y
    
    def loss(self, x, t) :
        y = self.predict(x)

        return cross_entropy_error(y, t) # predict로 나온 출력에 대해 교차 엔트로피 에러를 구한다.
    
    def accuracy(self, x, t) :
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t) :
        loss_W = lambda W : self.loss(x, t)
        
        grads = {}
        grads['W1'] = common.gradient.numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = common.gradient.numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = common.gradient.numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = common.gradient.numerical_gradient(loss_W, self.params['b2'])
        
        return grads

if __name__ == "__main__" :
    
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
    
    train_loss_list = []
    
    # 하이퍼 파라미터
    iters_num = 10000
    train_size = x_train.shape[0]
    batch_size = 100
    learning_rate = 0.1
    
    net = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)
    
    # print(net.params['W1'].shape) # (784, 100)
    # print(net.params['b1'].shape) # (100,)
    # print(net.params['W2'].shape) # (100, 10)
    # print(net.params['b2'].shape) # (10,)
    
    for i in range(iters_num) :
        batch_mask = np.random.choice(train_size, batch_size) # 훈련 데이터중에서 batch size만큼 랜덤으로 선택
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]
        
        # 기울기 계산
        grad = net.numerical_gradient(x_batch, t_batch)
        
        # 매개변수 갱신
        for key in ('W1', 'b1', 'W2', 'b2') :
            net.params[key] -= learning_rate * grad[key]
            
        # 학습 경과 기록
        loss = net.loss(x_batch, t_batch)
        train_loss_list.append(loss)
        
        
    print(train_loss_list)

KeyboardInterrupt: 